In [25]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import scipy.sparse as sp
import tensorflow as tf
from sklearn.model_selection import KFold
from tqdm import tqdm
import os


In [2]:
data = pd.read_csv("u.data", sep ="\t" , header = None ).sort_values([0,3]).reset_index(drop = True)

In [3]:
data[0] = data[0] - 1

In [4]:
data[1] = data[1] - 1

In [5]:
user_p = pd.read_csv("u.user" , sep = "|"  ,header = None)

In [6]:
user_p = user_p.drop(columns = [0])

In [7]:
user_p[2] = pd.Categorical(user_p[2]).codes

In [8]:
user_p[3] = pd.Categorical(user_p[3]).codes

In [9]:
user_p.head()

,1,2,3,4
0,24,1,19,85711
1,53,0,13,94043
2,23,1,20,32067
3,24,1,19,43537
4,33,0,13,15213


In [10]:
user_G = user_p[2]
user_J = user_p[3]
user_A = user_p[1]

In [11]:
user_num = 943
item_num = 1682

In [12]:
train_mat_x = sp.lil_matrix((943,1682) , dtype = np.int8)
train_mat_y = sp.lil_matrix((943,1682) , dtype = np.int8)

In [13]:
for i in tqdm(range(user_num)):
    u = data[data[0] == i]
    limit = int(u.shape[0]*0.8)
#     print(u.head())
#     print(u.iloc[0])
    for j in range(u.shape[0]):
        row = u.iloc[j][0]
        col = u.iloc[j][1]
        rate =u.iloc[j][2]
        if j < limit:
            train_mat_x[row,col] = rate
        else:
            train_mat_y[row,col] = rate
#     break

100%|████████████████████████████████████████| 943/943 [00:46<00:00, 20.48it/s]


In [14]:
train_mat_x = train_mat_x.toarray()

In [15]:
train_mat_y = train_mat_y.toarray()

In [16]:
x_buy_profile = tf.placeholder(tf.float32, [None,1682])

y_user_age = tf.placeholder(tf.float32 , [None,1])

y_user_gender = tf.placeholder(tf.int32 , [None,1])

y_user_job = tf.placeholder(tf.int32)

y_user_gender_one = tf.one_hot(y_user_gender,2)

y_user_job_one = tf.one_hot(y_user_job,20)

y_user_buy = tf.placeholder(tf.float32, [None,1682])

w1 = tf.get_variable('w1',shape=[1682, 512],initializer=tf.glorot_uniform_initializer())

b1 = tf.get_variable('b1',shape=[512],initializer=tf.zeros_initializer)

w2 = tf.get_variable('w2',shape=[512, 256],initializer=tf.glorot_uniform_initializer())

b2 = tf.get_variable('b2',shape=[256],initializer=tf.zeros_initializer)

# w3 = tf.get_variable('w3',shape=[1024, 512],initializer=tf.glorot_uniform_initializer())

# b3 = tf.get_variable('b3',shape=[512],initializer=tf.zeros_initializer)

# w4 = tf.get_variable('w4',shape=[512, 256],initializer=tf.glorot_uniform_initializer())

# b4 = tf.get_variable('b4',shape=[256],initializer=tf.zeros_initializer)

h1 = tf.matmul(x_buy_profile, w1) + b1

h1_sigmoid = tf.sigmoid(h1)

h2 = tf.matmul(h1_sigmoid, w2) + b2

h2_sigmoid = tf.sigmoid(h2)

# h3 = tf.matmul(h2_sigmoid, w3) + b3

# h3_sigmoid = tf.sigmoid(h3)

# h4 = tf.matmul(h3_sigmoid, w4) + b4

split1 , split2 = tf.split(h2_sigmoid, [128, 128], 1)

## split 1 do what
w5 = tf.get_variable('w5',shape=[128, 256],initializer=tf.glorot_uniform_initializer())

b5 = tf.get_variable('b5',shape=[256],initializer=tf.zeros_initializer)

w6 = tf.get_variable('w6',shape=[256, 512],initializer=tf.glorot_uniform_initializer())

b6 = tf.get_variable('b6',shape=[512],initializer=tf.zeros_initializer)

w7 = tf.get_variable('w7',shape=[512, 1682],initializer=tf.glorot_uniform_initializer())

b7 = tf.get_variable('b7',shape=[1682],initializer=tf.zeros_initializer)

# w8 = tf.get_variable('w8',shape=[1024, 2048],initializer=tf.glorot_uniform_initializer())

# b8 = tf.get_variable('b8',shape=[2048],initializer=tf.zeros_initializer)

# w9 = tf.get_variable('w9',shape=[2048, items],initializer=tf.glorot_uniform_initializer())

h5 = tf.matmul(split1, w5) + b5

h5_sigmoid = tf.sigmoid(h5)

h6 = tf.matmul(h5_sigmoid, w6) + b6

h6_sigmoid = tf.sigmoid(h6)

y_buy = tf.matmul(h6_sigmoid, w7) + b7

# h7_sigmoid = tf.sigmoid(h7)

# h8 = tf.matmul(h7_sigmoid, w8) + b8

# h8_sigmoid = tf.sigmoid(h8)

# y_items = tf.matmul(h8_sigmoid ,w9)

## split 2 do what

## split3 for JOB

## split4 for Gender

## split5 for Age
split3 , split4 , split5 = tf.split(split2,[80,16,32] ,1)

##split3 part

w10 = tf.get_variable('w10',shape=[80, 64],initializer=tf.glorot_uniform_initializer())

b10 = tf.get_variable('b10',shape=[64],initializer=tf.zeros_initializer)

w11 = tf.get_variable('w11',shape=[64, 32],initializer=tf.glorot_uniform_initializer())

b11 = tf.get_variable('b11',shape=[32],initializer=tf.zeros_initializer)

w12 = tf.get_variable('w12',shape=[32, 20],initializer=tf.glorot_uniform_initializer())

b12 = tf.get_variable('b12',shape=[20],initializer=tf.zeros_initializer)

h10 = tf.matmul(split3, w10) + b10

h10_sigmoid = tf.sigmoid(h10)

h11 = tf.matmul(h10_sigmoid, w11) + b11

h11_sigmoid = tf.sigmoid(h11)

y_job = tf.matmul(h11_sigmoid, w12) + b12

## to crossentropy for JOB

##split4 part

w13 = tf.get_variable('w13',shape=[16, 8],initializer=tf.glorot_uniform_initializer())

b13 = tf.get_variable('b13',shape=[8],initializer=tf.zeros_initializer)

w14 = tf.get_variable('w14',shape=[8, 4],initializer=tf.glorot_uniform_initializer())

b14 = tf.get_variable('b14',shape=[4],initializer=tf.zeros_initializer)

w15 = tf.get_variable('w15',shape=[4, 2],initializer=tf.glorot_uniform_initializer())

b15 = tf.get_variable('b15',shape=[2],initializer=tf.zeros_initializer)

h13 = tf.matmul(split4, w13) + b13

h13_sigmoid = tf.sigmoid(h13)

h14 = tf.matmul(h13_sigmoid, w14) + b14

h14_sigmoid = tf.sigmoid(h14)

y_gender = tf.matmul(h14_sigmoid, w15) + b15

## to crossentropy for GENDER

w16 = tf.get_variable('w16',shape=[32, 16],initializer=tf.glorot_uniform_initializer())

b16 = tf.get_variable('b16',shape=[16],initializer=tf.zeros_initializer)

w17 = tf.get_variable('w17',shape=[16, 8],initializer=tf.glorot_uniform_initializer())

b17 = tf.get_variable('b17',shape=[8],initializer=tf.zeros_initializer)

w18 = tf.get_variable('w18',shape=[8, 1],initializer=tf.glorot_uniform_initializer())

b18 = tf.get_variable('b18',shape=[1],initializer=tf.zeros_initializer)

h16 = tf.matmul(split5, w16) + b16

h16_sigmoid = tf.sigmoid(h16)

h17 = tf.matmul(h16_sigmoid, w17) + b17

h17_sigmoid = tf.sigmoid(h17)

y_age = tf.matmul(h17_sigmoid, w18) + b18

## to mae for Age

cross_entropy_job =tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_user_job_one, logits=y_job))

cross_entropy_gender =tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_user_gender_one, logits=y_gender))

mae_age = tf.reduce_mean(tf.abs(tf.subtract(y_user_age, y_age)))

mae_buy = tf.reduce_mean(tf.abs(tf.subtract(y_user_buy, y_buy)))

profile_loss = (cross_entropy_job + cross_entropy_gender + mae_age)


train_step_profile = tf.train.GradientDescentOptimizer(0.5).minimize(profile_loss)

train_step_buy = tf.train.GradientDescentOptimizer(0.5).minimize(mae_buy)

sess = tf.InteractiveSession()

tf.global_variables_initializer().run()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [19]:
epoch = 10000
for i in tqdm(range(epoch)):
    batch_size = 400
    train_id = np.random.choice(range(train_mat_x.shape[0]), size=batch_size)
    train_x_buy_profile = train_mat_x[train_id]
    train_y_user_buy =  train_mat_y[train_id]
    
    train_y_user_age = np.array(user_A[train_id]).reshape((-1,1))
    train_y_user_gender = np.array(user_G[train_id]).reshape((-1,1))
    train_y_user_job =np.array(user_J[train_id]).reshape((-1,1))
#     print(train_x_buy_profile.shape)
#     print(train_y_user_buy.shape)
#     print(train_y_user_age.shape)
#     print(train_y_user_gender.shape)
#     print(train_y_user_job.shape)
    sess.run(train_step_buy, feed_dict={x_buy_profile: train_x_buy_profile , y_user_buy:train_y_user_buy , y_user_age:train_y_user_age, y_user_gender:train_y_user_gender ,y_user_job:train_y_user_job})
    sess.run(train_step_profile , feed_dict={x_buy_profile: train_x_buy_profile , y_user_buy:train_y_user_buy , y_user_age:train_y_user_age, y_user_gender:train_y_user_gender ,y_user_job:train_y_user_job})
#     break


  1%|▍                                     | 101/10000 [00:19<31:03,  5.31it/s]


  2%|▊                                     | 203/10000 [00:38<30:50,  5.29it/s]


  3%|█▏                                    | 305/10000 [00:54<25:31,  6.33it/s]


  4%|█▌                                    | 407/10000 [01:11<26:08,  6.11it/s]


  5%|█▉                                    | 509/10000 [01:29<26:33,  5.96it/s]


...

 97%|███████████████████████████████████▊ | 9689/10000 [28:33<01:12,  4.31it/s]


 98%|████████████████████████████████████▏| 9791/10000 [28:52<00:37,  5.62it/s]


 99%|████████████████████████████████████▌| 9893/10000 [29:12<00:19,  5.42it/s]


100%|████████████████████████████████████▉| 9995/10000 [29:31<00:00,  5.63it/s]


100%|████████████████████████████████████| 10000/10000 [29:32<00:00,  4.93it/s]

In [23]:
save = tf.train.Saver()

In [54]:
test_id = np.random.choice(range(train_mat_x.shape[0]), size=batch_size)
test_x_buy_profile = train_mat_x[test_id]
test_y_user_buy =  train_mat_y[test_id]    
test_y_user_age = np.array(user_A[test_id]).reshape((-1,1))
test_y_user_gender = np.array(user_G[test_id]).reshape((-1,1))
test_y_user_job =np.array(user_J[test_id]).reshape((-1,1))

In [60]:
embedding = sess.run(h2_sigmoid, feed_dict={x_buy_profile: test_x_buy_profile , y_user_buy:test_y_user_buy , y_user_age:test_y_user_age, y_user_gender:test_y_user_gender ,y_user_job:test_y_user_job})
# sess.run(, feed_dict={x_buy_profile: train_x_buy_profile , y_user_buy:train_y_user_buy , y_user_age:train_y_user_age, y_user_gender:train_y_user_gender ,y_user_job:train_y_user_job})

In [62]:
embedding

array([[5.3797752e-01, 5.0087947e-01, 5.9258544e-01, ..., 2.6759291e-01,
        8.5768700e-03, 8.7359869e-01],
       [5.8911687e-01, 5.3633827e-01, 5.2042758e-01, ..., 5.1990688e-02,
        4.8051089e-02, 2.4454445e-02],
       [4.8014924e-01, 5.0024849e-01, 4.7889018e-01, ..., 2.5290400e-02,
        8.2603544e-02, 4.0914059e-02],
       ...,
       [4.9342650e-01, 5.7563794e-01, 6.1228120e-01, ..., 1.1492878e-01,
        2.8941035e-04, 9.5766854e-01],
       [4.3213561e-01, 5.1707137e-01, 5.2970642e-01, ..., 2.9172897e-03,
        1.1014342e-03, 3.5953522e-04],
       [5.6109923e-01, 4.5419568e-01, 4.6890882e-01, ..., 1.2920618e-02,
        1.3140547e-01, 8.1811130e-02]], dtype=float32)

In [36]:
save_path = save.save(sess,"save/model.ckpt")  